In [1]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import numpy as np 
import warnings 
pd.options.mode.chained_assignment = None

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
# set white seaborn plotting style 
sns.set(context='talk', style='white', rc={'figure.facecolor':'white'}, font_scale=1)
sns.set_style('ticks')

# metadata

In [ ]:
metadata = pd.read_csv("combined_metadata_redone.csv")

# microbiome data

In [ ]:
uq_micro = pd.read_csv("uq_micro_read_counts.csv")

uq_micro = uq_micro.dropna(subset="Taxa")

uq_micro = uq_micro.T.reset_index()
uq_micro.columns = uq_micro.iloc[0]
uq_micro = uq_micro[1:]

uq_micro = uq_micro.rename(columns={"Taxa":"sample"})

In [ ]:
uq_micro["sample"] = uq_micro["sample"].astype(int)
uq_micro["sample"] = uq_micro["sample"] + 96

In [ ]:
uq_micro_meta = uq_micro.merge(metadata, left_on="sample", right_on="sample_num")

In [ ]:
uq_micro_meta["Sex"] = uq_micro_meta["Sex"].replace({"Male":0, "Female":1})
uq_micro_meta["status"] = uq_micro_meta["sample_type"].replace({"Control":0, "ALS":1, "BS Control":0})

# association testing

In [ ]:
tests = []

for micro in list(uq_micro)[1:]: 
    
    df = uq_micro_meta[[micro, "status", "Sex", "age", "concentration", "postOtr"]]
    df[micro] = df[micro].astype(int)
    df = df.rename(columns={micro:"micro"})


    mod = smf.glm(formula='status ~ micro + Sex + age + concentration', 
                  family=sm.families.Binomial(), 
                  data=df).fit()  

    pval = mod.summary2().tables[1]["P>|z|"]["micro"]
    coeff = mod.summary2().tables[1]["Coef."]["micro"]

    tests.append([micro, pval, coeff])



In [ ]:
uq_micro_test = pd.DataFrame(tests, columns=["microorganism", "pval", "coefficient"])

In [ ]:
uq_micro_test["fdr"] = multipletests(uq_micro_test["pval"], method="fdr_bh", alpha=0.05)[0]
uq_micro_test["qval"] = multipletests(uq_micro_test["pval"], method="fdr_bh", alpha=0.05)[1]

In [ ]:
uq_micro_test.sort_values(by="pval")

In [ ]:
uq_micro_test[uq_micro_test["fdr"] == True]